In [3]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd

In [4]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [5]:
print(pd.__version__)

0.24.2


### Load \& clean up the data set 
- Drop columns where the majority of the values are missing
- Remove unwanted columns & rows

In [6]:
data = pd.read_csv('LoanStats3a.csv', skiprows=1) # skipping row 1 so pandas can parse the data properly
thresh_count = len(data) * .2
data = data.dropna(thresh=thresh_count,axis=1) # Drop columns with > 80% of the values missing
data = data.drop(['emp_title', 'emp_length', 'home_ownership', 'desc' ],axis=1)   # Don't need these columns.
data = data[(data.loan_status == 'Fully Paid') | (data.loan_status == 'Charged Off') | 
            (data.loan_status == 'Current') | (data.loan_status == 'Late')]
data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens,hardship_flag,debt_settlement_flag
0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,NaN,3.0,0.0,13648.0,83.7%,9.0,f,0.0,0.0,5863.155187,5833.84,5000.00,863.16,0.00,0.0,0.00,Jan-2015,171.62,Dec-2018,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
1,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,NaN,3.0,0.0,1687.0,9.4%,4.0,f,0.0,0.0,1014.530000,1014.53,456.46,435.17,0.00,122.9,1.11,Apr-2013,119.66,Oct-2016,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
2,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,NaN,2.0,0.0,2956.0,98.5%,10.0,f,0.0,0.0,3005.666844,3005.67,2400.00,605.67,0.00,0.0,0.00,Jun-2014,649.91,Jun-2017,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
3,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,35.0,10.0,0.0,5598.0,21%,37.0,f,0.0,0.0,12231.890000,12231.89,10000.00,2214.92,16.97,0.0,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N
4,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,80000.0,Source Verified,Dec-2011,Fully Paid,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,38.0,15.0,0.0,27783.0,53.9%,38.0,f,0.0,0.0,4066.908161,4066.91,3000.00,1066.91,0.00,0.0,0.00,Jan-2017,67.30,Apr-2018,0.0,1.0,Individual,0.0,0.0,0.0,0.0,0.0,N,N


### AVG. INTEREST RATE
Calculate weighted average interest rate on issued loans 

In [7]:
# Convert the values in the 'int_rate' column to type float
data.loc[:, 'int_rate'] = data.int_rate.str.rstrip('%').astype(float)

Make a new grade column combining F and G classes as one.

In [8]:
data.loc[:, 'grade_altered'] = data['grade'] # Create new col 'grade_altered', containing the same values as 'grade'
data.loc[data.grade.isin(['F', 'G']), 'grade_altered'] = 'FG' # In col 'grade_altered', replace 'F' & 'G' with 'FG'
data.grade_altered.value_counts().sort_index() # Return the new grade counts

A     10085
B     12035
C      8111
D      5325
E      2858
FG     1372
Name: grade_altered, dtype: int64

In [9]:
# Calculate the weighted loan amount; the weighting factor is the interest rate
data.loc[:, 'weight_loan_amt'] = data.loan_amnt * data.int_rate

In [10]:
# function to calculate the weighted interest rate
def calc_avg_int_rate(grp):
    return (grp.weight_loan_amt.sum()/grp.loan_amnt.sum())


In [11]:
avg_int_rates = data.groupby('grade_altered').apply(lambda x: pd.Series({'AVG. INTEREST RATE':
                    calc_avg_int_rate(x)}))
avg_int_rates.loc['All'] = avg_int_rates.mean()
avg_int_rates

,AVG. INTEREST RATE
grade_altered,
A,7.378612
B,11.088584
C,13.601470
D,15.861973
E,17.927385
FG,20.339580
All,14.366267


### Number of loans issued
- Total number of loans
- Number of loans 'Fully Paid'
- Number of loans 'Charged Off'

In [12]:
num_of_loans = data.groupby(['grade_altered', 'loan_status']).apply(lambda v: v.shape[0]).reset_index()
num_of_loans = num_of_loans.pivot(index='grade_altered',
       columns='loan_status',
       values=0)

total_loans_issued = pd.DataFrame(num_of_loans.sum(axis=1),
                   columns=['Total Issued']
                   )

num_of_loans = total_loans_issued.join(num_of_loans)

num_of_loans.loc['All'] = num_of_loans.sum()

num_of_loans.join(avg_int_rates)




,Total Issued,Charged Off,Fully Paid,AVG. INTEREST RATE
grade_altered,,,,
A,10085,602,9483,7.378612
B,12035,1433,10602,11.088584
C,8111,1356,6755,13.601470
D,5325,1130,4195,15.861973
E,2858,725,2133,17.927385
FG,1372,424,948,20.339580
All,39786,5670,34116,14.366267


## Dollar amount

In [18]:
fully_paid = data[data.loan_status == 'Fully Paid'].groupby('grade_altered').agg({
                                    'loan_amnt': 'sum',  
                                    'total_rec_int': 'sum',
                                    'total_rec_prncp': 'sum'

                               })

fully_paid = fully_paid.rename(columns={
    'loan_amnt':'FULLY PAID',
    'total_rec_int': 'INTEREST PAYMENTS RECEIVED',
    'total_rec_prncp': 'PRINCIPAL PAYMENTS RECEIVED', 
    
})

charged_off = data[data.loan_status == 'Charged Off'].groupby('grade_altered').agg({
                                    'loan_amnt': 'sum',  
                                    'total_rec_prncp': 'sum'

                               })

charged_off = charged_off.rename(columns={
    'loan_amnt':'LOAN_AMT',
    'total_rec_prncp': 'PRINCIPAL PAYMENTS RECEIVED' 
    
})

charged_off.loc[:, 'CHARGED OFF (NET)'] = charged_off['LOAN_AMT'] - charged_off['PRINCIPAL PAYMENTS RECEIVED']

dollar_amt_df = fully_paid.join(charged_off['CHARGED OFF (NET)']) \
    .join(data.groupby('grade_altered')['loan_amnt'].sum().reset_index().rename(columns={'loan_amnt': 'TOTAL ISSUED'}
                                                                               ).set_index('grade_altered'))

dollar_amt_df.loc['All', :] = dollar_amt_df.sum()


dollar_amt_df = dollar_amt_df[['TOTAL ISSUED', 'FULLY PAID',  'CHARGED OFF (NET)', 
                               'PRINCIPAL PAYMENTS RECEIVED', 'INTEREST PAYMENTS RECEIVED']]
dollar_amt_df

,TOTAL ISSUED,FULLY PAID,CHARGED OFF (NET),PRINCIPAL PAYMENTS RECEIVED,INTEREST PAYMENTS RECEIVED
grade_altered,,,,,
A,86982400.0,82286850.0,2660250.00,8.014777e+07,8118504.75
B,133872900.0,118213175.0,9743755.53,1.156015e+08,20313074.49
C,89358850.0,74329450.0,9958390.41,7.288295e+07,16820668.21
D,65438225.0,51623850.0,9344092.60,5.084205e+07,14050388.20
E,45373750.0,33830250.0,8151473.84,3.254727e+07,11379047.71
FG,25824775.0,17691050.0,6045480.79,1.711373e+07,7188351.16
All,446850900.0,377974625.0,45903443.17,3.691353e+08,77870034.52


### \% of issued dollars

In [19]:
total_issued_divisor = dollar_amt_df['TOTAL ISSUED'].to_dict()
round(dollar_amt_df.apply(lambda v: v/total_issued_divisor[v.name], axis=1)*100, 2)

,TOTAL ISSUED,FULLY PAID,CHARGED OFF (NET),PRINCIPAL PAYMENTS RECEIVED,INTEREST PAYMENTS RECEIVED
grade_altered,,,,,
A,100.0,94.60,3.06,92.14,9.33
B,100.0,88.30,7.28,86.35,15.17
C,100.0,83.18,11.14,81.56,18.82
D,100.0,78.89,14.28,77.69,21.47
E,100.0,74.56,17.97,71.73,25.08
FG,100.0,68.50,23.41,66.27,27.84
All,100.0,84.59,10.27,82.61,17.43
